In [ ]:
import os
import findspark

os.environ["SPARK_HOME"] = "D:/spark-3.5.5-bin-hadoop3"
findspark.init()

from pyspark.sql import SparkSession
spark = SparkSession.builder.master('local[*]').getOrCreate()


import pandas as pd

In [ ]:
PANDAS = pd.read_csv('COLUNA_.csv', sep=';')
SPARK = spark.read.csv('COLUNA_.csv', sep=';', inferSchema=True)

OBS: os Displays abaixo são apenas para mostrar todos os prints no mesmo output

In [ ]:
# O PRINT DO DATAFRAME DO SPARK É TOTALMENTE DIFERENTE, PORÉM ELE POSSUI UMA FUNÇÃO ESPECIFICA PARA FICAR COMO O DF DO PANDAS:
display(PANDAS)
SPARK.toPandas()

In [ ]:
display(PANDAS.head(5))
SPARK.limit(5).toPandas()

In [ ]:
PANDAS.rename(columns={'TESTE': 'COLUNA_1'}, inplace=True)
SPARK.withColumRenamed('TESTE', 'COLUNA_1')

In [ ]:
print(PANDAS.info)
SPARK.printSchema()

In [ ]:
from pyspark.sql.types import DoubleType, StringType
from pyspark.sql import functions as f

PANDAS['COLUNA_2'] = PANDAS['COLUNA_2'].str.replace(',', '.')
SPARK = SPARK.withColumn('COLUNA_2', f.regexp_replace('COLUNA_2', ',', '.'))

In [ ]:
PANDAS['COLUNA_2'] = PANDAS['COLUNA_2'].astype(float)
SPARK = SPARK.withColumn('COLUNA_2', SPARK['COLUNA_2'].cast(DoubleType()))

In [ ]:
# ASSUMINDO QUE O CAMPO DE DATA ESTA COMO INT/LONG, ETC EX 20200924 -> 24/09/2020
PANDAS['COLUNA_3'] = pd.to_datetime(PANDAS['COLUNA_3'], dayfirst=True, errors='coerce')
SPARK = SPARK.withColumn('COLUNA_3', f.to_date(SPARK.COLUNA_3.cast(StringType()), 'yyyMMdd'))#.withColum ...

In [ ]:
# SEMELHANTE AO SELECT SQL
PANDAS[['COLUNA_1, COLUNA_2, COLUNA_3']]
SPARK.select('*').show(5, truncate=False)

In [ ]:
# O DADO DEVE ESTAR EM FORMATO DATE
PANDAS['COLUNA_4'] = PANDAS['COLUNA_3'].dt.year
SPARK.select(f.year('COLUNA_3').alias('COLUNA_4'))

In [ ]:
PANDAS[(pd.isnull(PANDAS['COLUNA_3'])) | (pd.isnull(PANDAS['COLUNA_4']))]
SPARK.select(f.when(f.isnull('COLUNA_3')))

In [ ]:
# ELE LEVA EM CONSIDERAÇÃO A TIPAGEM 
PANDAS.fillna(0)
SPARK.na.fill(0)

# PARA NONE:
PANDAS.fillna('-')
SPARK.na.fill('-')